In [1]:
from torchvision import datasets, transforms
import torch
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import random_split, DataLoader

from cnn_model import CNNModel
from vit_model import ViTModel

In [2]:
# we'll see if needed
# Define transforms to preprocess the data (you can customize these as needed)
transform = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
    transforms.Pad((111, 96, 112, 96), fill=0),  # Add Padding:  Our images are 64x33 (left,right,top, bottom)
    transforms.ToTensor()              # Convert the image to a PyTorch tensor
])

In [3]:
# Load the dataset using ImageFolder
dataset = datasets.ImageFolder(root="dataset", transform=transform)
dataset.class_to_idx

{'holo': 0, 'non-holo': 1}

In [4]:
dataset[0][0].shape

torch.Size([3, 256, 256])

In [5]:
# Use random_split to split the dataset into train, validation, and test sets
train_dataset, validation_dataset, test_dataset = random_split(dataset, [0.8, 0.1, 0.1])

In [6]:
# Define DataLoader for each split using the respective sampler
training_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True )
validation_dataloader = DataLoader(validation_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [7]:
len(train_dataset), len(validation_dataset), len(test_dataset)

(24000, 3000, 3000)

In [8]:
#model = CNNModel(num_classes=1,
#                 lr=0.0001)
model = ViTModel(num_classes=1,
                 lr=0.0001)

/home/yanis/paris_cite/S2/TER/model_training/venv/lib/python3.12/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_tiny_patch16_224_in21k to current vit_tiny_patch16_224.augreg_in21k.
  model = create_fn(


AttributeError: 'VisionTransformer' object has no attribute 'classifier'

In [ ]:
model

CNNModel(
  (pretrained_mobilenet): MobileNetV3(
    (conv_stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): Hardswish()
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (bn1): BatchNormAct2d(
            16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): ReLU(inplace=True)
            (conv_expand): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
            (gate): Hardsigmoid()
          )
          (conv_pw): Conv2d(16, 8, kernel_si

In [11]:
checkpoint_callback = ModelCheckpoint(
        dirpath="models",
        monitor="validation_accuracy",
        filename="best",
        mode="max",
        save_last=True,
        verbose=True
    )
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = L.Trainer(max_epochs=30,
                    log_every_n_steps=1,
                    val_check_interval=0.25,
                    callbacks=[checkpoint_callback])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
trainer.fit(model=model, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

You are using a CUDA device ('NVIDIA GeForce RTX 4060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: /home/yanis/paris_cite/S2/TER/model_training/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                 | Type           | Params
--------------------------------------------------------
0 | pretrained_mobilenet | MobileNetV3    | 569 K 
1 | accuracy_metric      | BinaryAccuracy | 0     
--------------------------------------------------------
569 K     Trainable params
0         Non-trainable params
569 K     Total params
2.277     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/yanis/paris_cite/S2/TER/model_training/venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/yanis/paris_cite/S2/TER/model_training/venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  3.72it/s]

/home/yanis/paris_cite/S2/TER/model_training/venv/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


...

Epoch 17, global step 3384: 'validation_accuracy' was not in top 1


Epoch 18:  25%|██▌       | 47/188 [00:11<00:35,  3.94it/s, v_num=0, train_accuracy_step=0.773, validation_loss=0.752, validation_accuracy=0.623, train_accuracy_epoch=0.777] 

Epoch 18, global step 3431: 'validation_accuracy' was not in top 1


Epoch 18:  50%|█████     | 94/188 [00:24<00:24,  3.83it/s, v_num=0, train_accuracy_step=0.844, validation_loss=0.608, validation_accuracy=0.700, train_accuracy_epoch=0.777]

Epoch 18, global step 3478: 'validation_accuracy' was not in top 1
/home/yanis/paris_cite/S2/TER/model_training/venv/lib/python3.12/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
print(f"Loading model best.ckpt")
best_model = CNNModel.load_from_checkpoint("models/best.ckpt", num_classes=1, lr=0.0001).to('cuda')
best_model

Loading model best.ckpt


CNNModel(
  (pretrained_mobilenet): MobileNetV3(
    (conv_stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): Hardswish()
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (bn1): BatchNormAct2d(
            16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): ReLU(inplace=True)
            (conv_expand): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
            (gate): Hardsigmoid()
          )
          (conv_pw): Conv2d(16, 8, kernel_si

In [ ]:
input, label = next(iter(test_dataloader))

In [ ]:
input = input.to('cuda')
input.shape

torch.Size([128, 3, 256, 256])

In [ ]:
label = label.to('cuda')
label.shape

torch.Size([128])

In [ ]:
# For every INPUT_SIZE / Prediction size do the prediction then append
best_model.eval()
with torch.no_grad():
        #input = input.unsqueeze(0).to('cpu')
        #print(input)
        y_hat = best_model(input).squeeze()
        result = torch.nn.functional.sigmoid(y_hat)
# Output the last 100 of the result

In [ ]:
result

tensor([0.0112, 0.6729, 0.4799, 0.5394, 0.6935, 0.3679, 0.6721, 0.6628, 0.1597,
        0.2371, 0.3803, 0.1504, 0.6274, 0.1340, 0.6385, 0.8761, 0.4273, 0.6579,
        0.4732, 0.7325, 0.5150, 0.9165, 0.2742, 0.8953, 0.8226, 0.6522, 0.7563,
        0.0218, 0.6132, 0.9433, 0.8861, 0.6816, 0.5239, 0.2946, 0.3296, 0.8952,
        0.6044, 0.6107, 0.8360, 0.8750, 0.1774, 0.4725, 0.1984, 0.2591, 0.6267,
        0.7675, 0.7917, 0.8536, 0.7096, 0.4436, 0.4027, 0.5656, 0.6997, 0.0962,
        0.7740, 0.4534, 0.1998, 0.2822, 0.6989, 0.6886, 0.8878, 0.4022, 0.8913,
        0.2914, 0.0381, 0.1421, 0.5100, 0.4969, 0.7570, 0.9035, 0.2806, 0.8596,
        0.0125, 0.8994, 0.7339, 0.6969, 0.3651, 0.6381, 0.2822, 0.2750, 0.1011,
        0.0651, 0.4816, 0.1114, 0.2325, 0.2459, 0.3862, 0.8329, 0.7881, 0.6440,
        0.5890, 0.4002, 0.5563, 0.1724, 0.5859, 0.2708, 0.6225, 0.4646, 0.8356,
        0.4161, 0.2011, 0.5262, 0.5730, 0.8126, 0.1203, 0.0204, 0.2079, 0.2474,
        0.3807, 0.7265, 0.4311, 0.0364, 

In [ ]:
label

tensor([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0], device='cuda:0')